In [ ]:
# import sys
# from pathlib import Path

# # Get the project root path and add it to sys.path
# project_root = Path().resolve().parent  # Adjust to point to your project root if necessary
# src_path = project_root / 'src'

# if str(src_path) not in sys.path:
#     sys.path.append(str(src_path))

In [1]:
import pandas as pd
import numpy as np

from src.data_preprocessing import DataPreprocessing
from src.data_loader import DataLoader
from src.model_training import ModelTraining
from src.model_inference import ModelInference

In [4]:
# get the data
train_id = DataLoader("data/train_identity.csv")
train_trans = DataLoader("data/train_transaction.csv")

test_id = DataLoader("data/test_identity.csv")
test_trans = DataLoader("data/test_transaction.csv")

In [5]:
# preprocess the data
data_preprocessing = DataPreprocessing()
X_train, y_train = data_preprocessing.transform(train_id.dataset, train_trans.dataset)

Merged DataFrame shape: (144233, 434)


In [8]:
params = {
    'loss_function': 'Logloss',
    'eval_metric': 'f1',
    'random_seed': 42
}

fit_params = {
    'cat_features': ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9'],
    'early_stopping_rounds': 100
}

model = ModelTraining(params=params)
model.train(X_train, y_train, fit_params=fit_params)

ValueError: 'M1' is not in list